## Machine Learning Record Mining

Project to create a pipeline that uses GeoDeepDive's output to find Unaquired Sites for Neotoma.

Using NLP parsed text and a Data Science approach, identify whether a paper is suitable for Neotoma and detect features such as 'Site Name', 'Location', 'Age Span' and 'Site Descriptions'.

In [1]:
# Loading libraries

import numpy as np
import pandas as pd
import csv
import psycopg2
import re
import ast

# Load Postgres Server
from src.config import config

In [2]:
# Options for DF display
pd.set_option('display.max_colwidth', 50)
pd.set_option('display.max_rows', 10)

#pd.set_option('display.max_rows', None)
#pd.set_option('display.max_columns', None)
#pd.set_option('display.width', None)
#pd.set_option('display.max_colwidth', -1)

## Loading and viewing the Data

### Loading NLP Sentences

In [3]:
# Connect to PostgreSQL server from terminal:
# pg_ctl -D PSQL_Data -l logfile start

try:
    params = config()
    # Connect to the PostgreSQL database
    conn = psycopg2.connect(**params)
    # Create a new cursor
    cur = conn.cursor()
    nlp_sentences = pd.read_sql_query('''SELECT * FROM sentences;''', conn)
    # Close the cursor and connection to so the server can allocate
    # bandwidth to other requests
    cur.close()
    conn.close()
    print('Data obtained from PostgreSQL')
    
# If no SQL db, load from a file
except:
    header_list = ["_gddid", "sentid", "wordidx", "words", "part_of_speech", "special_class", 
               "lemmas", "word_type", "word_modified"]
    nlp_sentences = pd.read_csv("../Do_not_commit_data/sentences_nlp352", sep='\t', names = header_list)
    nlp_sentences = nlp_sentences.replace('"', '', regex = True)\
                                 .replace('\{', '', regex = True)\
                                 .replace('}', '', regex = True)\
                                 .replace(',', ',', regex = True)
    nlp_sentences['wordidx']= nlp_sentences['wordidx'].str.split(",")
    nlp_sentences['words']= nlp_sentences['words'].str.split(",")
    nlp_sentences['poses']= nlp_sentences['poses'].str.split(",")
    nlp_sentences['ners']= nlp_sentences['ners'].str.split(",")
    nlp_sentences['lemmas']= nlp_sentences['lemmas'].str.split(",")
    nlp_sentences['dep_paths']= nlp_sentences['dep_paths'].str.split(",")
    nlp_sentences['dep_parents']= nlp_sentences['dep_parents'].str.split(",")
    print('Data obtained from text file')

Data obtained from PostgreSQL


In [4]:
nlp_sentences.head(3)

,_gddid,sentid,wordidx,words,part_of_speech,special_class,lemmas,word_type,word_modified
0,54b43266e138239d8684efed,1,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","[Available, online, at, www.sciencedirect.com,...","[JJ, NN, IN, NNP, NNP, NNP, CD, -LRB-, CD, -RR...","[O, O, O, O, O, O, NUMBER, O, DATE, O, NUMBER,...","[available, online, at, www.sciencedirect.com,...","[dep, dep, dep, dep, dep, dep, dep, , dep, , d...","[218, 218, 218, 218, 218, 218, 218, 0, 218, 0,..."
1,54b43266e138239d8684efed,2,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]","[The, Chihuahueños, Bog, record, extends, to, ...","[DT, NNP, NN, NN, VBZ, TO, IN, CD, JJ, NN, NN, .]","[O, O, O, O, O, O, O, NUMBER, O, DURATION, O, O]","[the, Chihuahueños, bog, record, extend, to, o...","[det, compound, compound, nsubj, , case, amod,...","[4, 4, 4, 5, 0, 11, 11, 11, 11, 11, 5, 0]"
2,54b43266e138239d8684efed,3,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","[An, Artemisia, steppe, ,, then, an, open, Pic...","[DT, NNP, NN, ,, RB, DT, JJ, NNP, NN, VBD, IN,...","[O, O, O, O, O, O, O, LOCATION, LOCATION, O, O...","[a, Artemisia, steppe, ,, then, a, open, Picea...","[det, compound, nsubj, , advmod, det, amod, co...","[3, 3, 10, 0, 9, 9, 9, 9, 10, 0, 14, 14, 14, 1..."


In [5]:
# Running this takes forever. Do not run unless necessary
# nlp_sentences.describe(include = 'all')

In [6]:
avg_sentences = nlp_sentences.groupby('_gddid')['_gddid','sentid'].transform('max')
avg_sentences.drop_duplicates(subset='_gddid', keep='first', inplace=True)
avg_sentences.describe()

,sentid
count,218.000000
mean,513.600917
std,232.460891
min,2.000000
25%,397.250000
50%,504.000000
75%,638.250000
max,1330.000000


From the Summary Statistics, despite having 111,965 sentences, there are only 218 unique articles (`_gddid`). The article with most sentences, had 1330 sentences; that's the max number found in `sentid`.  
In order to know the average of sentences, I grouped by `_gddid` and then, obtained the number of sentences per article. After that, I just got the mean through a Summary of statistics to know that on average, each article has 514 sentences.

### Loading Bibliography Data

In [7]:
import json
with open('../Do_not_commit_data/bibjson', 'r') as f:
    bib_dict = json.load(f)

In [8]:
# Normalizing data so that we have access to the 'identifier'

# TODO Load into SQL server and connect through SQL
bibliography = pd.io.json.json_normalize(bib_dict,'identifier',['publisher', 'title', ['journal','name', 'name'], ['author'], 'year', 'number', 'volumne', ['link'], '_gddid', 'type', 'pages'], record_prefix='_', errors='ignore')

bibliography['link'] = bibliography['link'].astype(str)

url = bibliography['link'].str.split(", ", expand = True)

bibliography['link_url'] = url[0]
bibliography['link_type'] = url[1]

bibliography['link_url'] = bibliography['link_url'].replace("\[{'url': '", "", regex = True)\
                                                   .replace("'", "", regex = True)

bibliography['link_type'] = bibliography['link_type'].replace("'type': '", "", regex = True)\
                                                     .replace("'}]", "", regex = True)

bibliography['author'] = bibliography['author'].astype(str)
bibliography['author'] = bibliography['author'].replace("\[{'name': '", "", regex = True)\
                                               .replace("{'name': '", "", regex = True)\
                                               .replace("'},", ";", regex = True)\
                                               .replace("'}]", "", regex = True)
bibliography = bibliography[['_type', '_id', 'publisher', 'title', 'journal.name.name',	'author', 'year', 'number', 'volumne', '_gddid', 'type', 'pages', 'link_url', 'link_type']]

bibliography.head(3)

,_type,_id,publisher,title,journal.name.name,author,year,number,volumne,_gddid,type,pages,link_url,link_type
0,doi,10.1016/S0277-3791(99)00007-4,Elsevier,"Palaeoclimate, chronology and vegetation histo...",Quaternary Science Reviews,"Huntley, B.; Watts, W.A.; Allen, J.R.M.; Zolit...",1999,7,NaN,550453fde1382326932d85f7,article,945--960,http://www.sciencedirect.com/science/article/p...,publisher
1,doi,10.1139/b92-002,Canadian Science Publishing,Holocene history of forest trees on the Bruce ...,Canadian Journal of Botany,"Bennett, K. D.",1992,1,NaN,578b5aabcf58f1587003efba,article,6--18,http://www.nrcresearchpress.com/doi/abs/10.113...,publisher
2,doi,10.1016/0031-0182(80)90043-7,Elsevier,Glacial sequence and environmental history in ...,"Palaeogeography, Palaeoclimatology, Palaeoecology","Van Der Hammen, T.; Barelds, J.; De Jong, H.; ...",1980,NaN,NaN,54b43244e138239d868493cd,article,247--340,http://www.sciencedirect.com/science/article/p...,publisher


In [9]:
bibliography.describe(include = 'all')

,_type,_id,publisher,title,journal.name.name,author,year,number,volumne,_gddid,type,pages,link_url,link_type
count,221,221,221,221,221,221,221,183,0,221,221,216,221,220
unique,1,221,6,221,41,213,47,28,0,221,1,214,221,1
top,doi,10.1016/j.quascirev.2011.06.010,Elsevier,Stable isotopes in yellow-bellied marmot (Marm...,Quaternary Research,"Markgraf, Vera",2011,3,NaN,55076c16e1382326932d9561,article,1--13,http://doi.wiley.com/10.1002/2017JG004015,publisher
freq,221,1,138,1,44,3,14,40,NaN,1,221,2,1,220


## EDA

Reviewing our data includes skimming through some papers online and seeing if the data is consistent with our NLP Sentences dataframe.

From there, we can also visualize what we would like our model to predict: 'Location', 'Site Name', 'Age Span', and 'Site Description' from a "Human perspective".

### Finding Latitude and Longitud via REGEX 

After reviewing some PDF's and viewing the data parsing, I noticed some possible patterns for coordinates.

<img src="figures/img/00_snippet_of_pdf.png" alt="Drawing" style="width: 300px;"/>

Which translates to our pandas dataframe as:

![alt text](figures/img/01_snippet_in_sql.png "PandasVersion")

As a first problem, we can notice that the degrees symbol `°` was translated as a `3`. On some other occassions, it is confused as an `o` or a middle point `◦`. For `'`, this symbol got confused by the word `cents`.

I am still going to use normal REGEX to find coordinates in the most intuitive way. 

A coordinate for longitude is a number ranging from -180° to 180°. It has three components, `°, ',"` and it covers the position in E or W.

Latitude goes from -90° to 90°, has the same three components and goes from S to N.

Using REGEX, and assuming data consistency, the following code should be able to extract coordinates. 

In [10]:
dms_regex = r"([-]?\d{1,3}[°|′|\'|,]{0,3}\d{1,2}[,|′|\']{1,3}\d{0,2}[,|′|\']{1,4}[NESWnesw]?[\s|,|\']+?[-]?\d{1,3}[°|,|′|\']+\d{1,2}[,|′|\']+\d{0,2}[,|′|\'][,|′|\']{0,4}[NESWnesw]?)"

dd_regex = "[\\{,][-]?[1]?[0-9]{1,2}\\.[0-9]{1,}[,]?[NESWnesw],"

In [11]:
# TODO Find REGEX of couples of numbers (123, 234)
nlp_sentences['words_as_string'] = nlp_sentences['words'].apply(lambda x: ','.join(map(str, x)))
nlp_sentences['dms_regex'] = nlp_sentences['words_as_string'].str.findall(dms_regex)


#nlp_sentences[['dms_regex', 'words']]


#nlp_sentences['dms_regex'] = nlp_sentences['dms_regex'].astype(str)
#nlp_sentences['dms_regex'] = nlp_sentences['dms_regex'].replace(",", ";", regex = True)#\
                                                       #.replace(",", "", regex = True)

nlp_sentences[['words_as_string', 'words', 'dms_regex']].head(3)

,words_as_string,words,dms_regex
0,"Available,online,at,www.sciencedirect.com,Quat...","[Available, online, at, www.sciencedirect.com,...",[]
1,"The,Chihuahueños,Bog,record,extends,to,over,15...","[The, Chihuahueños, Bog, record, extends, to, ...",[]
2,"An,Artemisia,steppe,,,then,an,open,Picea,woodl...","[An, Artemisia, steppe, ,, then, an, open, Pic...",[]


In [12]:
#nlp_sentences[['words_as_string', 'words', 'dms_regex']].describe()

The `dms_regex` only returned 95 unique values. That's to say that from the 111965 sentences, only 95 possess something similar to the regex we are looking for.  

Let's see an example of a regex straction as a sanity check that we are getting the values that we need. 

In [13]:
italy_example = nlp_sentences[nlp_sentences['_gddid'] == '550453fde1382326932d85f7']
italy_example = italy_example[italy_example['sentid'] == 10]
italy_example[['words', 'words_as_string', 'dms_regex']]

,words,words_as_string,dms_regex
94875,"[Introduction, Laghi, di, Monticchio, are, two...","Introduction,Laghi,di,Monticchio,are,two,maar,...","[403,56,40,N,,,153,36,48,E]"


### Problems with the REGEX approach

As seen before, extracting information using REGEX is very complicated. We would require a lot of queries that would retrieve each particular case.

A possible alternative could be using NER, Named Entity Recognition.

In [14]:
def order_article(article_id):
    '''
    Function to find an article by its gddid in the NLP sentences and have it displayed in order
    
    Keyword arguments:
    article_id -- gddid
    
    Returns:
    article ordered by sentence index
    '''
    article = nlp_sentences[nlp_sentences['_gddid'] == article_id]
    return article[['sentid', 'words']].sort_values(by = 'sentid')

Article 01  
Skimmed info: 
 * `Article Name:` Palaeoclimate_chronology_and_vegetation_history_of_the_Weichselian_Lateglacial
 * `Link`: http://www.sciencedirect.com/science/article/pii/S0277379199000074  
 * `Site Name`:  Laghi di Monticchio
 * `Location`:  40° 56' 40" N, 15° 36' 48" E
 * `Age Span`: 
 * `Site Descriptions`: The record from Lago Grande di Monticchio now adds macrofossil evidence of the presence of relatively boreal tree taxa during the Lateglacial interstadial and provides a chronology based upon annually laminated sediments.

In [12]:
# Article 01 
order_article('550453fde1382326932d85f7')

,sentid,words
94865,1,"[Quaternary, Science, Reviews, 18, -LRB-, 1999..."
94866,2,"[Plant, macrofossil, data, from, one, core, pr..."
94867,3,"[Quantitative, palaeoclimate, reconstructions,..."
94868,4,"[The, chronological, framework, for, Lateglaci..."
94870,5,"[Surface, conditions, in, the, North, Atlantic..."
...,...,...
95325,448,"[Vegetation, history, and, climate, of, the, l..."
95326,449,"[Quaternary, Science, Reviews, ,, 15, ,, 133, ..."
95327,450,"[Zolitschka, ,, B., ,, &, Negendank, ,, J., F...."
95329,451,"[Sedimentology, ,, dating, and, palaeoclimatic..."


Article 02  
Skimmed info:  
* `Article:` Development of the Mixed Connifer Forest in Northern New Mexico and its relationship to Holocene environmental change
* `Link`: http://www.sciencedirect.com/science/article/pii/S0033589407001512
* `Site Name`:  Chihuahuenos Bog 
* `Location`:  ??? No Given Coordinates 
* `Age Span`:   
* `Site Descriptions`:  Chihuahueños Bog is located within the mixed conifer forest and has affinities to the flora of the southern Rocky Mountains(Foxx and Tierney, 1985; Allen, 2004). Although the area immediately surrounding the bog has been heavily logged in recent years, nearly every conifer species growing in montaneforests of the Jemez mountains is represented either along theshore of the bog or in the surrounding uplands.  

In [13]:
# Article 2
order_article('54b43266e138239d8684efed')

,sentid,words
0,1,"[Available, online, at, www.sciencedirect.com,..."
1,2,"[The, Chihuahueños, Bog, record, extends, to, ..."
2,3,"[An, Artemisia, steppe, ,, then, an, open, Pic..."
3,4,"[C/N, ratios, ,, δ13C, and, δ15N, values, indi..."
4,5,"[Higher, percentages, of, aquatic, algae, and,..."
...,...,...
605,572,"[Is, the, Valles, caldera, entering, a, new, c..."
144,573,"[Geology, 23, ,, 411, --, 414, .]"
607,574,"[Wright, Jr., ,, H.E., ,, Bent, ,, A.M., ,, Ha..."
608,575,"[Present, and, past, vegetation, of, the, Chus..."


Article 03  
Skimmed info:  
* `Link`: http://dx.doi.org/10.1130/g35541.1  # No Full access to article
* `Site Name`:  Cahokia region, Illinois, USA
* `Location`:  N38°42'00" - N38°42'00", W90°04'60" - W90°04'60" - Not found on text (yet).
* `Age Span`:   
* `Site Descriptions`: Record from Horseshoe Lake, an oxbow lake in the central Mississippi River valley that is adjacent to the Cahokia site (Illinois, USA), the largest prehistoric settlement north of Mexico.   

In [14]:
# Article 3
order_article('57c5b941cf58f1338eaddb5b')

,sentid,words
67708,1,"[A, record, of, sustained, prehistoric, and, h..."
67709,2,"[Here, we, report, a, high-resolution, and, mu..."
67710,3,"[Palynological, and, carbon, isotope, data, do..."
67712,4,"[Rapid, forest, clearance, was, followed, clos..."
67713,5,"[Agricultural, intensiﬁcation, that, included,..."
...,...,...
67845,123,"[Simon, ,, M.L., ,, and, Parker, ,, K.E., ,, 2..."
67846,124,"[Smith, ,, B.D., ,, and, Yarnell, ,, R.A., ,, ..."
67848,125,"[Sugita, ,, S., ,, 1993, ,, A, model, of, poll..."
67849,126,"[Trubitt, ,, M.B.D., ,, 2000, ,, Mound, buildi..."


Article 04  
Skimmed info:  
* `Link:`http://www.tandfonline.com/doi/abs/10.1080/00173130902965157
* `Site Name`:  Begbunar
* `Location`:  42°09′ N, 22° 33′ E; 1750 ; in SQL file shows as: `42,°,09,cents,N,,,22,°,33,cents,E`
* `Age Span`:   
* `Site Descriptions`: Located in the central treeless zone of the Osogovo mountains, which are situated at the border between south-western Bulgaria and the north-eastern former Yugoslav Republic of Macedonia.

In [15]:
# Article 4
order_article('58d29193cf58f14928755ba5')

,sentid,words
110659,1,"[Grana, ISSN, :, 0017-3134, -LRB-, Print, -RRB..."
110660,2,"[Peat-bog, Begbunar, -LRB-, Osogovo, Mountains..."
110661,3,"[Peat-bog, Begbunar, -LRB-, Osogovo, Mountains..."
110662,4,"[Submit, your, article, to, this, journal, Art..."
110665,5,"[Peat-bog, Begbunar, -LRB-, Osogovo, Mountains..."
...,...,...
110740,80,"[Ann, .]"
110741,81,"[Sofia, Univ., .]"
110742,82,"[Fac, .]"
110743,83,"[Geol, .]"


In [16]:
# Try to see the exact location. Wont show last few words.
pd.DataFrame(order_article('58d29193cf58f14928755ba5').iloc[4])

,110665
sentid,5
words,"[Peat-bog, Begbunar, -LRB-, Osogovo, Mountains..."


Article 05  
Skimmed info:  
* `Link`: http://dx.doi.org/10.1130/0016-7606(1997)109<1306:tfalqp>2.3.co
* `Site Name`:  Cottonwood III, Cottonwood IV, Red Lady, Red Well, Splains
* `Location`:  38°49'50' 106°24'45', 38°49'50' 106°24'45', 38°52'50' 107°2'30', 38°52'50' 107°2'30',38°53'40' 107°3'15', 38°52'28' 107°05', 38°50' 107°4'30', 38°50' 107°4'30', 38°52' 107°2'30', 38°45' 106°50', 38°45' 106°50'
* `Age Span`:   17000 - 400
* `Site Descriptions`:   The lower timberline or lower forest border is the lower elevational limit of a continuous forest belt. In the Rocky Mountains it forms a sharp contrast with shrublands or grasslands in valley bottoms. Trees may extend below the lower timberline in favorable habitats, along streams or on rocky outcrops.  

In [17]:
# Article 5
order_article('57928e07cf58f133d1c26609')

,sentid,words
39606,1,"[Timberline, fluctuations, and, late, Quaterna..."
39607,2,"[By, tracking, climatically, sensitive, forest..."
39608,3,"[Pollen, data, suggest, that, prior, to, 11, 0..."
39609,4,"[The, inferred, climate, was, 2Ð5, ¡, C, coole..."
39611,5,"[Abies, -LRB-, fir, -RRB-, increased, in, abun..."
...,...,...
40107,448,"[Weber, ,, W., A., ,, 1987, ,, Colorado, flora..."
40108,449,"[Whitlock, ,, C., ,, 1993, ,, Postglacial, veg..."
40110,450,"[Whitlock, ,, C., ,, and, Bartlein, ,, P., J.,..."
40111,451,"[Wright, ,, H., E., ,, Jr., ,, 1983, ,, Late-Q..."


In [18]:
# Print the exact line where locations are.
pd.DataFrame(order_article('57928e07cf58f133d1c26609').iloc[179])

,39801
sentid,180
words,"[LOCATION, AND, AGE, OF, FOSSIL, STUDY, SITES,..."


In [19]:
regex_example = nlp_sentences[nlp_sentences['_gddid'] == '57928e07cf58f133d1c26609']
regex_example = regex_example[regex_example['sentid'] == 180]
regex_example[['words', 'words_as_string', 'dms_regex']]

,words,words_as_string,dms_regex
39801,"[LOCATION, AND, AGE, OF, FOSSIL, STUDY, SITES,...","LOCATION,AND,AGE,OF,FOSSIL,STUDY,SITES,Core,Lo...","[38,°,49,′,50,′,′,106,°,24,′,45,′,′,, 38,°,49,..."


In [20]:
# Find the article in the bibliography by gddid to retrieve Article Name

bibliography[bibliography['_gddid'] == '57928e07cf58f133d1c26609']

,_type,_id,publisher,title,journal.name.name,author,year,number,volumne,_gddid,type,pages,link_url,link_type
93,doi,10.1130/0016-7606(1997)109<1306:tfalqp>2.3.co;2,GSA,Timberline fluctuations and late Quaternary pa...,Geological Society of America Bulletin,"Fall, Patricia L.",1997,10,NaN,57928e07cf58f133d1c26609,article,1306--1320,http://dx.doi.org/10.1130/0016-7606(1997)109<1...,publisher


## Linking Neotoma DB to extract locations

In [21]:
neotoma = pd.read_csv("../Do_not_commit_data/data-1590729612420.csv")
neotoma = neotoma[['siteid', 'sitename', 'longitudeeast', 'latitudenorth', 'longitudewest', 'latitudesouth', 'sitedescription', 'doi']]

neotoma.head(3)

,siteid,sitename,longitudeeast,latitudenorth,longitudewest,latitudesouth,sitedescription,doi
0,10330,Lac du Sommet,-70.66468,47.71662,-70.66573,47.71382,"The small shallow Lac du Sommet (0.02 km2, 4 m...",10.1177/0959683611400199
1,10330,Lac du Sommet,-70.66468,47.71662,-70.66573,47.71382,"The small shallow Lac du Sommet (0.02 km2, 4 m...",10.1177/0959683611400199
2,1729,Myrtle Lake,-93.37853,47.98645,-93.39207,47.97876,Lake surrounded by peatland. Physiography: Red...,10.1139/b68-190


In [22]:
nlp_bib = nlp_sentences.merge(bibliography, on = '_gddid')
nlp_bib = nlp_bib.rename(columns={'_id': 'doi'})
nlp_bib.head(3)

,_gddid,sentid,wordidx,words,part_of_speech,special_class,lemmas,word_type,word_modified,words_as_string,...,title,journal.name.name,author,year,number,volumne,type,pages,link_url,link_type
0,54b43266e138239d8684efed,1,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","[Available, online, at, www.sciencedirect.com,...","[JJ, NN, IN, NNP, NNP, NNP, CD, -LRB-, CD, -RR...","[O, O, O, O, O, O, NUMBER, O, DATE, O, NUMBER,...","[available, online, at, www.sciencedirect.com,...","[dep, dep, dep, dep, dep, dep, dep, , dep, , d...","[218, 218, 218, 218, 218, 218, 218, 0, 218, 0,...","Available,online,at,www.sciencedirect.com,Quat...",...,Development of the mixed conifer forest in nor...,Quaternary Research,"Anderson, R. Scott; Jass, Renata B.; Toney, Ja...",2008,2,NaN,article,263--275,http://www.sciencedirect.com/science/article/p...,publisher
1,54b43266e138239d8684efed,2,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]","[The, Chihuahueños, Bog, record, extends, to, ...","[DT, NNP, NN, NN, VBZ, TO, IN, CD, JJ, NN, NN, .]","[O, O, O, O, O, O, O, NUMBER, O, DURATION, O, O]","[the, Chihuahueños, bog, record, extend, to, o...","[det, compound, compound, nsubj, , case, amod,...","[4, 4, 4, 5, 0, 11, 11, 11, 11, 11, 5, 0]","The,Chihuahueños,Bog,record,extends,to,over,15...",...,Development of the mixed conifer forest in nor...,Quaternary Research,"Anderson, R. Scott; Jass, Renata B.; Toney, Ja...",2008,2,NaN,article,263--275,http://www.sciencedirect.com/science/article/p...,publisher
2,54b43266e138239d8684efed,3,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","[An, Artemisia, steppe, ,, then, an, open, Pic...","[DT, NNP, NN, ,, RB, DT, JJ, NNP, NN, VBD, IN,...","[O, O, O, O, O, O, O, LOCATION, LOCATION, O, O...","[a, Artemisia, steppe, ,, then, a, open, Picea...","[det, compound, nsubj, , advmod, det, amod, co...","[3, 3, 10, 0, 9, 9, 9, 9, 10, 0, 14, 14, 14, 1...","An,Artemisia,steppe,,,then,an,open,Picea,woodl...",...,Development of the mixed conifer forest in nor...,Quaternary Research,"Anderson, R. Scott; Jass, Renata B.; Toney, Ja...",2008,2,NaN,article,263--275,http://www.sciencedirect.com/science/article/p...,publisher


In [23]:
#print(nlp_bib.shape)

(111965, 24)


In [24]:
nlp_bib[['words_as_string', 'journal.name.name', 'dms_regex']].head(3)

,words_as_string,journal.name.name,dms_regex
0,"Available,online,at,www.sciencedirect.com,Quat...",Quaternary Research,[]
1,"The,Chihuahueños,Bog,record,extends,to,over,15...",Quaternary Research,[]
2,"An,Artemisia,steppe,,,then,an,open,Picea,woodl...",Quaternary Research,[]


In [192]:
by_doi = neotoma.groupby('doi')
neotoma_sitenames = by_doi.apply(lambda x: list(x['sitename']))
neotoma_siteid = by_doi.apply(lambda x: list(x['siteid']))
neotoma_longeast = by_doi.apply(lambda x: list(x['longitudeeast']))
neotoma_latnorth = by_doi.apply(lambda x: list(x['latitudenorth']))

neotoma_joined_df = neotoma_sitenames.to_frame('sitenames').join(neotoma_siteid.to_frame('siteid')).join(neotoma_longeast.to_frame('longeast')).join(neotoma_latnorth.to_frame('latnorth'))
neotoma_joined_df = neotoma_joined_df.reset_index()

neotoma_joined_df.head(5)

,doi,sitenames,siteid,longeast,latnorth
0,0.1139/e80-122,"[East Baltic Bog, Mermaid Bog, Portage Bog]","[763, 1684, 1903]","[-62.15, -63.025, -64.075]","[46.408333, 46.25, 46.673611]"
1,10.1002/(SICI)1099-1417(199607/08)11:4<327::AI...,[Lundin Tower],[3256],[-2.97206],[56.21529]
2,10.1002/(SICI)1099-1417(199701/02)12:1<15::AID...,"[Spruce Pond, Spruce Pond, Spruce Pond]","[2508, 2508, 2508]","[-74.18236, -74.18236, -74.18236]","[41.237770000000005, 41.237770000000005, 41.23..."
3,10.1002/(SICI)1099-1417(199801/02)13:1<43::AID...,"[Pantano de Genagra, Pantano de Genagra]","[14030, 14030]","[-76.60428, -76.60428]","[2.4751, 2.4751]"
4,10.1002/1099-1417(200010)15:7<687::AID-JQS551>...,"[Laguna Loma Linda, Laguna Loma Linda]","[13971, 13971]","[-73.35661, -73.35661]","[3.30805, 3.30805]"


From Neotoma DB, we have obtained a dataframe grouped by DOI that lists each `sitename`, `siteid`, `longitude east`, and `latitunde north`. Now, let's join this information to the NLP_Bib dataframe. For now, let's do to the one that coordinates have been obtained with REGEX just to see if some of the numbers and sitenames make sense. 

In [193]:
nlp_bib_neotoma = nlp_bib.merge(neotoma_joined_df, on = 'doi')
print(nlp_bib_neotoma.shape)

(106640, 28)


In [194]:
# Filter df to just frames that has REGEX longer than one
mask = (nlp_bib_neotoma['dms_regex'].str.len() > 0)
nlp_bib_neotoma_regex = nlp_bib_neotoma.loc[mask]
nlp_bib_neotoma_regex[['words','dms_regex']].head(100)

print(nlp_bib_neotoma_regex.shape)

(93, 28)


In [195]:
nlp_bib_neotoma_regex = nlp_bib_neotoma_regex[['_gddid', 'sentid', 'doi', 'words', 'words_as_string', 'dms_regex', 'sitenames', 'latnorth', 'longeast']]
nlp_bib_neotoma_regex = nlp_bib_neotoma_regex.reset_index()
nlp_bib_neotoma_regex.head(1)

,index,_gddid,sentid,doi,words,words_as_string,dms_regex,sitenames,latnorth,longeast
0,3314,578a1f6ecf58f1b54b810cd9,94,10.1139/b85-070,"[Lake, Hope, Simpson, -LRB-, 52, ', 27, ', N, ...","Lake,Hope,Simpson,-LRB-,52,',27,',N,,,56,',26,...","[52,',27,',N,,,56,',26,',W]","[Lake Hope Simpson, 18 [Moraine Lake]]","[52.45, 52.266667]","[-56.433333, -58.05]"


We can see that most of the REGEX indeed pulled out the same locations for latitude and longitudes as in the Neotoma database.

In [196]:
nlp_bib_neotoma[['_gddid', 'sentid', 'doi', 'words', 'dms_regex', 'sitenames', 'latnorth', 'longeast']].head(2)

,_gddid,sentid,doi,words,dms_regex,sitenames,latnorth,longeast
0,54b43266e138239d8684efed,1,10.1016/j.yqres.2007.12.002,"[Available, online, at, www.sciencedirect.com,...",[],"[Chihuahueños Bog, Chihuahueños Bog]","[36.0491, 36.0491]","[-106.50963, -106.50963]"
1,54b43266e138239d8684efed,2,10.1016/j.yqres.2007.12.002,"[The, Chihuahueños, Bog, record, extends, to, ...",[],"[Chihuahueños Bog, Chihuahueños Bog]","[36.0491, 36.0491]","[-106.50963, -106.50963]"


In [244]:
nlp_bib_neotoma['latnorth_s'] = nlp_bib_neotoma['latnorth'].apply(lambda x: [int(i) for i in x])
nlp_bib_neotoma['latnorth_s'] = nlp_bib_neotoma['latnorth_s'].apply(lambda x: [str(i) for i in x])

nlp_bib_neotoma['longeast_s'] = nlp_bib_neotoma['longeast'].apply(lambda x: [int(i) for i in x])
nlp_bib_neotoma['longeast_s'] = nlp_bib_neotoma['longeast_s'].apply(lambda x: [str(i) for i in x])

In [250]:
# nlp_bib_neotoma[['latnorth_s', 'longeast_s']].head(-2)

In [252]:
nlp_bib_neotoma['intersection_words_lat'] = nlp_bib_neotoma[['words','latnorth_s']].apply(lambda x : list(set.intersection(*map(set,list(x)))),axis=1)
nlp_bib_neotoma['intersection_words_long'] = nlp_bib_neotoma[['words','longeast_s']].apply(lambda x : list(set.intersection(*map(set,list(x)))),axis=1)
df = nlp_bib_neotoma[(nlp_bib_neotoma['intersection_words_lat'].apply(len) > 0) & (nlp_bib_neotoma['intersection_words_long'].apply(len) > 0 )]
df2 = nlp_bib_neotoma[nlp_bib_neotoma['intersection_words_long'].apply(len) > 0 ]
print(df2.shape)
df2[['words', 'longeast_s', 'latnorth_s', 'intersection_words_lat', 'intersection_words_long']]



(219, 32)


,words,longeast_s,latnorth_s,intersection_words_lat,intersection_words_long
5261,"[Holarctic, Ecology, 2, ,, 65, --, 74, .]","[65, 65]","[68, 68]",[],[65]
8446,"[Site, details, The, alder-carr, fen, Na, Bahne, -LRB-, 240, m, a., s., l., ,, 50, °, 11, ', 51.4, '', N, ,, 15, °, 57, ', 32.9, '', E, ,, 1, ha, -RRB-, lies, below, the, youngest, ,, Würmian, terrace, in, a, floodplain, ,, 150, m, from, the, Orlice, River, ,, about, 8, km, east, of, the, Elbe, River, .]","[15, 15]","[50, 50]",[50],[15]
9273,"[Table, 2, Radiocarbon, age, measurements, Depth, -LRB-, cm, -RRB-, Material, Dry, weight, Laboratory, ;, -LRB-, mg, -RRB-, sample, number, 28, --, 29, 41, --, 42, 63, --, 64, 77, --, 78, 116, --, 117, 122, --, 123, 127, --, 128, Betula, twig, ,, woody, fragments, and, Empetrum, leaves, Betula, twig, ,, woody, fragments, and, Empetrum, leaves, Woody, fragments, ,, Empetrum, leaves, and, leaf, fragments, Woody, fragments, and, Empetrum, leaves, Twig, ,, woody, fragments, ,, Empetrum, and, Juniperus, leaves, ,, leaf, fragments, Empetrum, leaves, and, plant, debris, Woody, fragments, ,, Empetrum, leaves, and, leaf, fragments, 4.5, 3.7, 6.7, 15.5, 13.2, 6.3, ...]","[28, 28, 28, 28]","[71, 71, 71, 71]",[],[28]
9790,"[Computers, &, Geosciences, 28, ,, 421, --, 426, .]","[28, 28, 28, 28]","[71, 71, 71, 71]",[],[28]
11223,"[The, sediments, contain, well-preserved, remains, of, plants, ,, diatoms, and, chironomids, and, indicate, :, -LRB-, 1, -RRB-, a, mosaic, pioneer, vegetation, of, dwarf-shrub, tundra, ,, snow, bed, vegetation, and, xerophytes, on, wind-blown, ridges, ,, -LRB-, 2, -RRB-, an, icefree, lake, during, summers, with, clear, water, at, low, pH, due, to, little, inwash, ,, -LRB-, 3, -RRB-, a, continental, climate, with, summers, slightly, warmer, than, today, -LRB-, ca, 10, 1C, -RRB-, ,, and, most, probably, with, very, cold, winters, ,, according, to, indications, of, permafrost, ,, and, -LRB-, 4, -RRB-, a, hiatus/reduced, sedimentation, during, at, least, the, ﬁrst, half, of, the, ...]","[10, 10, 10]","[62, 62, 62]",[],[10]
11261,"[This, tree-line, roughly, follows, the, 10, 1C, July, T, isotherm, in, continental, areas, -LRB-, Odland, ,, 1996, -RRB-, .]","[10, 10, 10]","[62, 62, 62]",[],[10]
11280,"[Contour, interval, 40, m., ARTICLE, IN, PRESS, 1230, A., Paus, et, al., /, Quaternary, Science, Reviews, 25, -LRB-, 2006, -RRB-, 1228, --, 1246, Table, 1, Lithostratigraphy, of, the, Fla˚, fattjønn, core, Layer, no., 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, Depth, -LRB-, cm, -RRB-, 1306, --, 1445.5, 1445.5, --, 1446.5, 1446.5, --, 1452.4, 1452.4, --, 1453.7, 1453.7, --, 1455.2, 1455.2, --, 1456.8, 1456.8, --, 1458.8, 1458.8, --, 1459.8, 1459.8, --, 1465, 1465, --, 1470, 1470, --, 1550, 1550, --, 1554, Description, -LRB-, Troels-Smith, 1955, -RRB-, Ld2, 4, ,, Ag, +, Ld1, 1, ,, Tb1, 1, ...]","[10, 10, 10]","[62, 62, 62]",[],[10]
11288,"[Plant, macrofossils, and, pollen, Macrofossil, samples, ,, usually, 10, cm, slices, of, the, core, ,, were, rinsed, with, water, through, a, sieve, with, a, mesh, width, of, 125, mm, .]","[10, 10, 10]","[62, 62, 62]",[],[10]
11293,"[Phase, contrast/oil, immersion, objectives, -LRB-, 63, Â, /, 1.4, ,, 100, Â, /, 1.3, -RRB-, and, 10, Â, oculars, were, used, .]","[10, 10, 10]","[62, 62, 62]",[],[10]
11319,"[For, the, chironomids, ,, the, lower, samples, with, a, count, sum, less, than, 10, was, excluded, from, the, zoning, .]","[10, 10, 10]","[62, 62, 62]",[],[10]


In [253]:
df3 = nlp_bib_neotoma[nlp_bib_neotoma['intersection_words_lat'].apply(len) > 0 ]
print(df3.shape)
df3[['words', 'longeast_s', 'latnorth_s', 'intersection_words_lat', 'intersection_words_long']]

(634, 32)


,words,longeast_s,latnorth_s,intersection_words_lat,intersection_words_long
393,"[Quaternary, Research, 36, ,, 307, --, 321, .]","[-106, -106]","[36, 36]",[36],[]
681,"[INDEX, LAT, -LRB-, ◦, N, -RRB-, LONG, -LRB-, ◦, W, -RRB-, ha, m, mg, μg, /, L, μS, /, cm, CaCo3/L, mg/L, mg/L, mg/L, mg/L, mg/L, mg/L, μg, /, L, Acidiﬁcation, 1, Chocolate, 44, ◦, 38.3, 63, ◦, 37.3, 7.1, 13.0, 6.2, 2, 497.0, 1.5, 11.6, 76.7, 122.0, 26.4, 0.3, 0.4, 0.1, 2, First, Chain, 44, ◦, 38.3, 63, ◦, 38.3, 72.6, 13.9, 4.8, 2, 328.0, −, 1.1, 5.5, 48.1, 80.3, 18.3, 0.1, 0.9, 0.4, 3, Frenchman, 44, ◦, 41.8, 63, ◦, 34.5, 8.4, 1.5, 6.1, 12, 469.0, 0.6, 4, Major, 44, ◦, 43.7, 63, ◦, 28.9, 343.0, 65.0, 4.7, ...]","[-63, -63, -63, -63, -63, -63, -63, -63, -63, -63, -63, -63, -63, -63, -63, -63, -63, -63, -63, -63, -63, -63, -63, -63, -63, -63, -63, -63, -63, -63, -63, -63, -63, -63, -63, -63, -63, -63, -63, -63, -63, -63, -63, -63, -63, -63, -63, -63, -63, -63, -63, -63, -63, -63, -63, -63, -63, -63, -63, -63, -63, -63, -63, -63, -63, -63, -63, -63, -63, -63, -63, -63, -63, -63, -63, -63, -63, -63, -63, -63, -63, -63, -63, -63, -63, -63, -63, -63, -63, -63, -63, -63, -63, -63, -63, -63, -63]","[44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44]",[44],[]
1998,"[Ret, ', iew, of, Palaeobotany, und, Palynology, 44, ,, 277-292, .]","[-70, -74, -67, -91, -75, -91, -91, -91, -89, -91, -91]","[44, 43, 46, 45, 41, 45, 48, 45, 46, 48, 48]",[44],[]
2154,"[2, ,, T., 114N, ,, R., 26W, -LRB-, 44, '', 45, ', N., lat., ,, 93, '', 55, ', W., long, ., -RRB-]","[-93, -93, -93]","[44, 44, 44]",[44],[]
2660,"[For, pollen, analysis, ,, 113, subsamples, were, taken, from, EJ-3, providing, a, record, from, 0, to, 45.6, cm, depth, .]","[-89, -89]","[0, 0]",[0],[]
2677,"[For, each, of, these, samples, ,, their, Euclidean, distance, score, was, set, to, 0, .]","[-89, -89]","[0, 0]",[0],[]
2740,"[Four, periods, show, marked, declines, in, spore, representation, ,, from, ca., 150, to, 100, BC, ,, at, ca., 0, BC, ,, from, ca, .]","[-89, -89]","[0, 0]",[0],[]
2783,"[Modern, pollen, traps, on, El, Junco, and, mud, --, water, interface, samples, from, the, lake, document, low, percentages, -LRB-, 0, --, 4, %, -RRB-, of, Alternanthera, and, Acalypha, ,, and, these, presumably, reﬂect, upslope, transport, of, lowland, species, ,, rather, than, local, production, -LRB-, Collins, and, Bush, 2011, -RRB-, .]","[-89, -89]","[0, 0]",[0],[]
3314,"[Lake, Hope, Simpson, -LRB-, 52, ', 27, ', N, ,, 56, ', 26, ', W, ,, elevation, 295, m, -RRB-, lies, 20, km, southwest, of, Port, Hope, Simpson, in, a, steep-sided, eastwest-trending, valley, that, is, drained, by, the, St., Lewis, River, .]","[-56, -58]","[52, 52]",[52],[]
3318,"[The, lake, basin, itself, has, an, irregular, though, conical, bathymetry, ,, with, a, mean, depth, of, 3.5, m, and, a, maximum, depth, of, 12.5, m., Independence, Lake, -LRB-, 52, ', 33, ', N, ,, 56, ', 54, ', W, ,, elevation, 172, m, -RRB-, is, located, in, similar, terrain, 20, km, farther, west, along, the, Alexis, River, .]","[-56, -58]","[52, 52]",[52],[]


## Intersections Between Sitenames and NLP Sentences
Let's now extract Sitenames from the Neotoma database. For this, I will group by doi and then, list all sitenames that are mentioned in each doi.

In [63]:
nlp_bib_neotoma['words_l']  = nlp_bib_neotoma['words'].astype(str).str.lower().transform(ast.literal_eval)


In [64]:
nlp_bib_neotoma['sitenames_l'] = nlp_bib_neotoma['sitenames'].astype(str).str.lower().transform(ast.literal_eval)
nlp_bib_neotoma[['words_l', 'sitenames_l']].head(2)

,words_l,sitenames_l
0,"[available, online, at, www.sciencedirect.com,...","[chihuahueños bog, chihuahueños bog]"
1,"[the, chihuahueños, bog, record, extends, to, ...","[chihuahueños bog, chihuahueños bog]"


In [65]:
nlp_bib_neotoma['sitenames_l']

0           [chihuahueños bog, chihuahueños bog]
1           [chihuahueños bog, chihuahueños bog]
2           [chihuahueños bog, chihuahueños bog]
3           [chihuahueños bog, chihuahueños bog]
4           [chihuahueños bog, chihuahueños bog]
                           ...                  
106635    [peat-bog begbunar, peat-bog begbunar]
106636    [peat-bog begbunar, peat-bog begbunar]
106637    [peat-bog begbunar, peat-bog begbunar]
106638    [peat-bog begbunar, peat-bog begbunar]
106639    [peat-bog begbunar, peat-bog begbunar]
Name: sitenames_l, Length: 106640, dtype: object

In [102]:
nlp_bib_neotoma['intersection_words_sitenames'] = nlp_bib_neotoma[['words_l','sitenames_l']].apply(lambda x : list(set.intersection(*map(set,list(x)))),axis=1)
#nlp_bib_neotoma['intersection_words_sitenames']

df = nlp_bib_neotoma[nlp_bib_neotoma['intersection_words_sitenames'].apply(len) > 0 ]
df[['words_l', 'sitenames_l', 'intersection_words_sitenames', 'dms_regex']]


,words_l,sitenames_l,intersection_words_sitenames,dms_regex
681,"[index, lat, -lrb-, ◦, n, -rrb-, long, -lrb-, ...","[anderson lake, anderson lake, bayers, bayers,...",[bayers],[]
788,"[seven, lakes, -lrb-, bayers, ,, bell, ,, cran...","[anderson lake, anderson lake, bayers, bayers,...",[bayers],[]
2110,"[quaternary, research, 16, ,, 66-79, -lrb-, 19...","[norwood, norwood, norwood]",[norwood],[]
2111,"[and, h., e., wright, ,, jr., ~, *, department...","[norwood, norwood, norwood]",[norwood],[]
2120,"[the, geologic, and, ecologic, succession, at,...","[norwood, norwood, norwood]",[norwood],[]
...,...,...,...,...
94912,"[m, a, p, of, central, europe, showing, the, s...","[krumpa, krumpa, krumpa]",[krumpa],[]
95048,"[distribution, of, mollusc, assemblages, -lrb-...","[krumpa, krumpa, krumpa]",[krumpa],[]
95138,"[1983, -rrb-, ,, the, krumpa, a, l, l, e, r, ~...","[krumpa, krumpa, krumpa]",[krumpa],[]
106480,"[kožlí, -lrb-, s., bohemia, ,, czech, republic...","[kožlí, kožlí]",[kožlí],[]


In [56]:
print(type(nlp_bib_neotoma['sitenames'][48]))
print(type(nlp_bib_neotoma['words'][48]))
print(type(nlp_bib_neotoma['words_as_string'][48]))

<class 'list'>
<class 'list'>
<class 'str'>
